In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
from decimal import Decimal

In [6]:
from time import time_ns
import numpy as np
import pandas as pd
from algorithms.golden_section_search import golden_section_search
from algorithms.successive_parabolic_interpolation import successive_parabolic_interpolation
from algorithms.brent import brent
from algorithms.bfgs import bfgs

## Task 2.1
Test function 

In [7]:
# initial settings
epsilon = 1e-05
type_optimization = 'min'
max_iter = 500

def f1(x): return -5 * x ** 5 + 4 * x ** 4 - 12 * x ** 3 + 11 * x ** 2 - 2 *x + 1
bounds_1 = (-0.5, 0.5)

def f2(x): return np.log(x - 2) ** 2 + np.log(10 - x) ** 2 - x ** 0.2 
bounds_2 = (6, 9.9)

def f3(x): return -3 * x * np.sin(0.75 * x) + np.exp(-2 * x)
bounds_3 = (0, 6.28) # 2 * np.pi)

def f4(x): return np.exp(3 * x) + 5 * np.exp(-2 * x)
bounds_4 = (0, 1)

def f5(x): return 0.2 * x * np.log(x) + (x - 2.3) ** 2
bounds_5 = (0.5, 2.5)

def func6(x):
    return (- x / (x ** 2 + 2))
bounds_6 = (1.3, 1.5)

def func7(x):
    return ((x + 0.004) ** 5 - 2 * (x + 0.004) ** 4)
bounds_7 = (1, 2)

def phi(alpha):
    if alpha <= 1 - 0.01:
        return 1 - alpha
    elif 1 - 0.01 <= alpha <= 1 + 0.01:
        return 1 / (2 * 0.01) * (alpha - 1) ** 2 + 0.01 / 2
    else:
        return alpha - 1
bounds_8 = (0.1, 2)

def func8(x):
    return (phi(x) + 2 * (1 - 0.01) / (39 * np.pi) * np.sin(39 * np.pi / 2 * x))

fs = [f1, f2, f3, f4, f5, func6, func7, func8]
bounds = [bounds_1, bounds_2, bounds_3, bounds_4, bounds_5, bounds_6, bounds_7, bounds_8]

In [8]:
test_data = pd.DataFrame(columns=['alg_name', 'f_name', 'bounds',
                                  'x_find', 'f_find', 
                                  'time', 'iteration'])

names = ['gss', 'ssi', 'brent', 'bfgs']
algs = [golden_section_search, successive_parabolic_interpolation, brent, bfgs]
i = 0

for j, (f, bound) in enumerate(zip(fs, bounds)):
    print(f'function_{j + 1}')
    for name, alg in zip(names, algs):
        
        if name != 'bfgs':
            time_start = time_ns()

            point, _ = alg(f, 
                           bound, 
                           epsilon=epsilon, 
                           type_optimization=type_optimization, 
                           max_iter=max_iter)

            time_alg = abs(time_ns() - time_start)
            _, history = alg(f, 
                             bound, 
                             epsilon=epsilon, 
                             type_optimization=type_optimization, 
                             max_iter=max_iter, 
                             keep_history=True)
            iter_alg = history['iteration'][-1]
            test_data.loc[i] = [name, 
                                f'f{j + 1}',
                                bound,
                                point['point'], 
                                point['f_value'],
                                time_alg, 
                                iter_alg]
            i += 1
        elif name == 'bfgs':
            time_start = time_ns()

            point, _ = alg(lambda x: f(x[0]), 
                           bound[1:], 
                           tolerance=epsilon, 
                           max_iter=max_iter)

            time_alg = abs(time_ns() - time_start)
            _, history = alg(lambda x: f(x[0]),  
                             bound[1:], 
                             tolerance=epsilon, 
                             max_iter=max_iter, 
                             keep_history=True)
            
            iter_alg = history['iteration'][-1]
            test_data.loc[i] = [name, 
                                f'f{j + 1}',
                                bound,
                                point['point'][0], 
                                point['f_value'][0],
                                time_alg, 
                                iter_alg]
            
            i += 1

function_1
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
function_2
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Out of bounds. code 1
Searching finished. Out of bounds. code 1
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
function_3
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Successfully. code 0
Searching finished. Su

C:\Users\vikto\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


In [9]:
pivot = test_data.round(2).pivot_table(index=['f_name', 'bounds'], columns='alg_name')

In [10]:
test_data

,alg_name,f_name,bounds,x_find,f_find,time,iteration
0,gss,f1,"(-0.5, 0.5)",0.109865,0.897633,0.000000,22
1,ssi,f1,"(-0.5, 0.5)",0.109860,0.897633,0.000000,8
2,brent,f1,"(-0.5, 0.5)",0.109860,0.897633,0.000000,11
3,bfgs,f1,"(-0.5, 0.5)",0.109860,0.897633,0.005000,3
4,gss,f2,"(6, 9.9)",8.501582,2.133838,0.001001,25
5,ssi,f2,"(6, 9.9)",7.950000,2.181961,0.000000,1
6,brent,f2,"(6, 9.9)",8.501577,2.133838,0.000000,10
7,bfgs,f2,"(6, 9.9)",8.501587,2.133838,0.003000,13
8,gss,f3,"(0, 6.28)",2.706478,-7.274358,0.001000,26
9,ssi,f3,"(0, 6.28)",2.706476,-7.274358,0.000000,7


In [13]:
pivot = pivot[['x_find', 'time', 'iteration', 'f_find']]
pivot

x_find                       time                      \
alg_name             bfgs brent   gss   ssi     bfgs   brent     gss ssi   
f_name bounds                                                              
f1     (-0.5, 0.5)   0.11  0.11  0.11  0.11  5596900  999000       0   0   
f2     (6, 9.9)      8.50  8.50  8.50  7.95  3000200  999900       0   0   
f3     (0, 6.28)     2.71  2.71  2.71  2.71  1999900  999900  998800   0   
f4     (0, 1)        0.24  0.24  0.24  0.24        0       0       0   0   
f5     (0.5, 2.5)    2.12  2.12  2.12  2.12        0       0       0   0   
f6     (1.3, 1.5)    1.41  1.41  1.41  1.41        0       0       0   0   
f7     (1, 2)        1.60  1.60  1.60  1.60        0       0       0   0   
f8     (0.1, 2)      1.00  1.00  1.00  1.00        0       0       0   0   

                   iteration               f_find                    
alg_name                bfgs brent gss ssi   bfgs brent   gss   ssi  
f_name bounds                                                        
f1     (-0.5, 0.5)         3    11  22   8   0.90  0.90  0.90  0.90  
f2     (6, 9.9)           13    10  25   1   2.13  2.13  2.13  2.18  
f3     (0, 6.28)           4    11  26   7  -7.27 -7.27 -7.27 -7.27  
f4     (0, 1)              7    10  22   5   5.15  5.15  5.15  5.15  
f5     (0.5, 2.5)          3     8  24   4   0.35  0.35  0.35  0.35  
f6     (1.3, 1.5)          4     8  19   5  -0.35 -0.35 -0.35 -0.35  
f7     (1, 2)              7    10  22   8  -2.62 -2.62 -2.62 -2.62  
f8     (0.1, 2)            2    10  24   9  -0.01 -0.01 -0.01 -0.01

In [18]:
# pivot.to_excel('pivot_algs.xlsx')